# Regresión Lineal

In [2]:
import tensorflow as tf, numpy as np, matplotlib.pyplot as plt

#### Cargando el data_set

In [7]:
data_set = np.load("proyecto_training_data.npy").astype(np.float32)
data_X1 = data_set[:,1] #OverallQual
data_X2 = data_set[:, 2] #1stFlrSF
data_Y = data_set[::,0] #SalesPrice

#### Del proyecto las variables seleccionadas fueron:

* OverallQual
* 1stFlrSF

### Grafo

In [9]:
tf.reset_default_graph()

grafo = tf.Graph()
with grafo.as_default():
    X = tf.placeholder("float", name = "X") 
    Y = tf.placeholder("float", name = "Y") 
    lr = tf.placeholder("float", name = "learning_rate")
    W = tf.Variable(0, name = "W", dtype="float")
    b = tf.Variable(0, name = "b", dtype="float")
    with tf.name_scope("y_hat") as scope:
        y_hat = tf.multiply(X, W) + b
    with tf.name_scope("costo") as scope:
        cost = tf.reduce_mean(tf.pow(y_hat - Y, 2))*0.5
    cost_summary = tf.summary.scalar("cost", cost)
    optimizer = tf.train.GradientDescentOptimizer(lr).minimize(cost)
    init = tf.global_variables_initializer()

In [20]:
def regresion_lineal(data_x, data_y, l_r, t_epoch, display_epoch, name):
    with tf.Session(graph = grafo) as sess:
        writer = tf.summary.FileWriter("./graphs/lr_" + str(l_r) + "_" + name, sess.graph)
        sess.run(init)
        for epoch in range(t_epoch):
            sess.run(optimizer, feed_dict = {X : data_x, Y : data_Y, lr : l_r})
            if (epoch + 1) % 10 == 0:
                c_sum, costo, weights, bias = sess.run([cost_summary, cost, W, b],feed_dict = {X : data_x, Y : data_y, \
                                                                                               lr : l_r}) 
                writer.add_summary(c_sum, epoch + 1)
            if (epoch + 1) % display_epoch == 0:
                c_sum, costo, weights, bias = sess.run([cost_summary, cost, W, b],feed_dict = {X : data_x, Y : data_y, \
                                                                                               lr : l_r})
                print("Epoch: ", (epoch + 1), "Costo: ", costo, "W: ", weights, "b: ", bias)
                print("----------------------------------------------")
        writer.close()
        return [weights, bias]

## Hipotesis:
* Se escogieron varios valores de learning rate para cada prueba.
* Se espera que el cuando lr = 0.00001 y lr = 0.000001 el costo disminuya lentamente, por lo que se necesitaran más iteraciones.
* El valor optimo podría encontrarse en rango de lr = 0.001 y lr = 0.0001
* Cuando lr = 0.1 puede que sean pasos muy grandes.

In [25]:
resultado = []
learning_rates = [0.1, 0.01, 0.001, 0.0001, 0.00001, 0.000001]

### obteniendo resultados de cada experimento utilizando la variable OverallQual para X

In [29]:
for num, lrate in enumerate(learning_rates):
    resultado.append(regresion_lineal(data_X1, data_Y, lrate, 10000, 10000, str(num)))

Epoch:  10000 Costo:  nan W:  nan b:  nan
----------------------------------------------
Epoch:  10000 Costo:  1180484900.0 W:  45302.492 b:  -95352.26
----------------------------------------------
Epoch:  10000 Costo:  1276168000.0 W:  35664.664 b:  -33623.33
----------------------------------------------
Epoch:  10000 Costo:  1406334000.0 W:  30424.77 b:  -61.862965
----------------------------------------------
Epoch:  10000 Costo:  1432565800.0 W:  29226.336 b:  4071.5085
----------------------------------------------
Epoch:  10000 Costo:  9546637000.0 W:  9809.271 b:  1482.2653
----------------------------------------------


![Grafo](grafo.png)

![graficas](l_r.png)

## Conclusiones

* Al analizar el patrón de resultado del entrenamiento el valor 0.1 no converge debido a que da pasos muy grandes.

* Cuando lr = 0.01 se puede observar que el costo dismunuye con mucha rápidez dejandolo como el hiperparametro de preferencia para entrenar el modelo.

* Los valores por debajo de lr = 0.01 necesitan más iteraciones para disminuir el costo del modelo.